In [1]:
import torch
from kwisatzHaderach import KwisatzHaderach
from Kaisarion import Kaisarion
import json
import os
import tqdm
import numpy as np
from datagen import *
from trainer import Trainer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def euclidean_distance(a, b):
    return torch.sqrt(torch.sum((a - b)**2, dim=-1) + 1e-12)

def mean_distance(a, b):
    return torch.mean(euclidean_distance(a, b))

def loss_fn(pr_acc, gt_acc, num_neighbors):
    gamma = 0.5
    neighbor_scale = 1/100
    importance = torch.exp(neighbor_scale * num_neighbors) # removed minus sign to give more importance to particles with more neighbors
    importance = importance / torch.max(importance)
    euclidean_distances = euclidean_distance(pr_acc, gt_acc)
    if importance.size()[0] == 0:
        importance = 1.0
    return torch.mean(importance *
                        euclidean_distances**gamma)


In [3]:
def loss_fn(pr_acc, gt_acc, num_neighbors, gt_pos, pr_pos, pos_importance=1.0, gamma=1.0):
    neighbor_scale = 1/100
    importance = torch.exp(neighbor_scale * num_neighbors) # removed minus sign to give more importance to particles with more neighbors
    importance = importance / torch.max(importance)
    euclidean_distances = euclidean_distance(pr_acc, gt_acc)
    if importance.size()[0] == 0:
        importance = 1.0
    if gamma != 1.0:
        return torch.mean(importance *
                        euclidean_distances**gamma) + pos_importance * torch.mean(importance * euclidean_distance(pr_pos, gt_pos)**gamma), torch.mean(euclidean_distances)
    return torch.mean(importance *
                        euclidean_distances) + pos_importance * torch.mean(importance * euclidean_distance(pr_pos, gt_pos)), torch.mean(euclidean_distances)

In [5]:
# clear cuda cache

torch.cuda.empty_cache()

In [9]:
model = Kaisarion(calc_neighbors=False)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.7, verbose=True)

trainer = Trainer(loss_fn=mean_distance, batch_size=32, device='cpu', mode='past')

trainer.train(model=model, optimizer=optimizer, scheduler=scheduler, rounds=40, epochs_per_dataset=1, save_after=10,scenes_per_dataset=10)


Adjusting learning rate of group 0 to 1.0000e-03.
Loaded weights from model_0.pt
Round 0
Generating dataset with 1 scenes...
100%|██████████| 1/1 [00:49<00:00, 49.14s/it]
Epoch 0
  0%|          | 0/31 [00:00<?, ?it/s]


RuntimeError: Storage size calculation overflowed with sizes=[4478562642679330237]